<a href="https://colab.research.google.com/github/Lawrence-Krukrubo/EDA/blob/master/sql_for_data_analysis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><h1>SQL Joins...</h1></b>

We connect to Google CloudSQL and make analysis with the Patch and Posey Database.<br>

Thanks to this [article](https://towardsdatascience.com/sql-on-the-cloud-with-python-c08a30807661) for making the connection process clearer.

If we want to download the parch-and-posey.sql file to maybe upload to a database, use this [link](https://storage.googleapis.com/kaggle1980/parch.sql) to the updated file from cloud-storage.

In [1]:
# Next mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# set working directory to Udacity
%cd /content/gdrive/MyDrive/Colab_Notebooks/Udacity

/content/gdrive/MyDrive/Colab_Notebooks/Udacity


In [3]:
%ls

 aws_machine_learning_foundations/  'linear-example-data (1).xlsx'
 client-cert.pem                     Problem_Solving_w_Advanced_Analytics.ipynb
 client-key.pem                      server-ca.pem
 computer_vision/                    sql_for_data_analysis1.ipynb
 intro_to_algorithm.ipynb            statistics/
 intro_to_artificial_intelligence/   time_series_forecasting.ipynb
 intro_to_data_analysis/             Udac_Prog_Foundations_Python/
 intro_to_machine_learning.ipynb     version_control_with_git/
 linear_algebra_refresher.ipynb


In [4]:
!pip install mysql-connector-python

     |████████████████████████████████| 18.0MB 199kB/s 


In [5]:
import mysql.connector
from mysql.connector.constants import ClientFlag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

In [6]:
config = {
    'user': 'root',
    'password': 'root',
    'host': '35.226.26.66',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'server-ca.pem',
    'ssl_cert': 'client-cert.pem',
    'ssl_key': 'client-key.pem'
}

# now we establish our connection
try:
    cnxn = mysql.connector.connect(**config)
    print('Connection to CloudSQL Instance Successful!')
except Exception as e:
    print(e)

Connection to CloudSQL Instance Successful!


In [7]:
config

{'client_flags': [2048],
 'host': '35.226.26.66',
 'password': 'root',
 'ssl_ca': 'server-ca.pem',
 'ssl_cert': 'client-cert.pem',
 'ssl_key': 'client-key.pem',
 'user': 'root'}

Now we connect to parch_and_posey_db by adding database: parch_and_posey_db to our config dictionary and connecting just like we did before:

In [8]:
config['database'] = 'parch_and_posey_db'  # add new database to config dict
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

Let's see the first 3 data of the different tables in parch and posey database

In [9]:
# let's run the show tables command 

cursor.execute('show tables')
out = cursor.fetchall()
out

[('accounts',), ('orders',), ('region',), ('sales_reps',), ('web_events',)]

Defining a method that converts a select query to a data frame

In [10]:
def query_to_df(query):
    # so we never have more than 20 rows displayed
    pd.set_option('display.max_rows', 20) 
    df = None
    cursor.execute(query)
    columns = cursor.description
    result = []
    for value in cursor.fetchall():
        tmp = {}
        for (index,column) in enumerate(value):
            tmp[columns[index][0]] = [column]
        result.append(tmp)

    # Create a DataFrame from all results
    for ind, data in enumerate(result):
        if ind >= 1:
            x = pd.DataFrame(data)
            df = pd.concat([df, x], ignore_index=True)
        else:
            df = pd.DataFrame(data)

    return df

In [11]:
# 1. For the accounts table
query = 'SELECT * FROM accounts LIMIT 3'
query_to_df(query)

,id,name,website,lats,longs,primary_poc,sales_rep_id
0,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500
1,1011,Exxon Mobil,www.exxonmobil.com,41.169156,-73.849374,Sung Shields,321510
2,1021,Apple,www.apple.com,42.290495,-76.084009,Jodee Lupo,321520


In [12]:
# 2. For the orders table
query = 'SELECT * FROM orders LIMIT 3'
query_to_df(query)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18


In [13]:
# 3. For the region table
query = 'SELECT * FROM region LIMIT 3'
query_to_df(query)

,id,name
0,1,Northeast
1,2,Midwest
2,3,Southeast


In [14]:
# 1. For the web_events table
query = 'SELECT * FROM web_events LIMIT 3'
query_to_df(query)

,id,account_id,occurred_at,channel
0,1,1001,2015-10-06 17:13:58,direct
1,2,1001,2015-11-05 03:08:26,direct
2,3,1001,2015-12-04 03:57:24,direct


<h3>Overview</h3>

Writing Joins is the real strength and magic of SQL. Joins are used to read data from multiple tables to power your analysis.

<h3>Database Normalization</h3>

When creating a database, it is really important to think about how data will be stored. This is known as normalization, and it is a huge part of most SQL classes. If you are in charge of setting up a new database, it is important to have a thorough understanding of database normalization.

There are essentially three ideas that are aimed at database normalization:

* Are the tables storing logical groupings of the data?
* Can I make changes in a single location, rather than in many tables for the same information?
* Can I access and manipulate data quickly and efficiently?
This is discussed in detail [here](https://www.itprotoday.com/sql-server/sql-design-why-you-need-database-normalization).

<h3><b>Joins</b></h3>

The whole purpose of `JOIN` statements is to allow us to pull data from more than one table at a time.

Again - `JOINs` are useful for allowing us to pull data from multiple tables. This is both simple and powerful all at the same time.

With the addition of the `JOIN` statement to our toolkit, we will also be adding the `ON` statement.

We use `ON` clause to specify a `JOIN` condition which is a logical statement to combine the table in `FROM` and `JOIN` statements.

<h3>Join Statement Analysis</h3>

```
SELECT orders.*
FROM orders
JOIN accounts
ON orders.account_id = accounts.id;
```
The `SELECT` clause indicates which column(s) of data you'd like to see in the output (For Example, orders.* gives us all the columns in orders table in the output). The `FROM` clause indicates the first table from which we're pulling data, and the `JOIN` indicates the second table. The `ON` clause specifies the column on which you'd like to merge the two tables together.

In [15]:
query = 'SELECT orders.* FROM orders JOIN accounts \
        ON orders.account_id = accounts.id'
query_to_df(query)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18
3,4,1001,2016-01-02 01:18:24,144,32,0,176,718.56,239.68,0.00,958.24
4,5,1001,2016-02-01 19:27:27,108,29,28,165,538.92,217.21,227.36,983.49
...,...,...,...,...,...,...,...,...,...,...,...
6907,6908,4501,2016-06-29 04:03:39,11,199,59,269,54.89,1490.51,479.08,2024.48
6908,6909,4501,2016-07-29 19:58:32,5,91,96,192,24.95,681.59,779.52,1486.06
6909,6910,4501,2016-08-27 00:58:11,16,94,82,192,79.84,704.06,665.84,1449.74
6910,6911,4501,2016-11-22 06:52:22,63,67,81,211,314.37,501.83,657.72,1473.92


**What to Notice**

We are able to pull data from two tables:

* orders
* accounts

Above, we are only pulling data from the orders table since in the `SELECT` statement we only reference columns from the orders table.

The `ON` statement holds the two columns that get linked across the two tables. 

**Additional Information**

If we wanted to only pull individual elements from either the orders or accounts table, we can do this by using the exact same information in the `FROM` and `ON` statements. However, in your `SELECT` statement, you will need to know how to specify tables and columns in the `SELECT` statement:

The table name is always before the period.<br>
The column you want from that table is always after the period.
For example, if we want to pull only the account name and the dates in which that account placed an order, but none of the other columns, we can do this with the following query:

```
SELECT accounts.name, orders.occurred_at
FROM orders
JOIN accounts
ON orders.account_id = accounts.id;
```

In [16]:
query = 'SELECT accounts.name, orders.occurred_at FROM orders JOIN accounts ON \
        orders.account_id = accounts.id;'
query_to_df(query)

,name,occurred_at
0,Walmart,2015-10-06 17:31:14
1,Walmart,2015-11-05 03:34:33
2,Walmart,2015-12-04 04:21:55
3,Walmart,2016-01-02 01:18:24
4,Walmart,2016-02-01 19:27:27
...,...,...
6907,SpartanNash,2016-06-29 04:03:39
6908,SpartanNash,2016-07-29 19:58:32
6909,SpartanNash,2016-08-27 00:58:11
6910,SpartanNash,2016-11-22 06:52:22


This query only pulls two columns, not all the information in these two tables. Alternatively, the below query pulls all the columns from both the accounts and orders table.

```
SELECT *
FROM orders
JOIN accounts
ON orders.account_id = accounts.id;
```

In [17]:
query = 'SELECT * FROM orders JOIN accounts ON orders.account_id = accounts.id'
query_to_df(query)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd,name,website,lats,longs,primary_poc,sales_rep_id
0,1001,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500
1,1001,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500
2,1001,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500
3,1001,1001,2016-01-02 01:18:24,144,32,0,176,718.56,239.68,0.00,958.24,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500
4,1001,1001,2016-02-01 19:27:27,108,29,28,165,538.92,217.21,227.36,983.49,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6907,4501,4501,2016-06-29 04:03:39,11,199,59,269,54.89,1490.51,479.08,2024.48,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970
6908,4501,4501,2016-07-29 19:58:32,5,91,96,192,24.95,681.59,779.52,1486.06,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970
6909,4501,4501,2016-08-27 00:58:11,16,94,82,192,79.84,704.06,665.84,1449.74,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970
6910,4501,4501,2016-11-22 06:52:22,63,67,81,211,314.37,501.83,657.72,1473.92,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970


**Quiz Questions**

1. Try pulling all the data from the accounts table, and all the data from the orders table.

2. Try pulling standard_qty, gloss_qty, and poster_qty from the orders table, and the website and the primary_poc from the accounts table.

In [20]:
# Try pulling all the data from the accounts table, and all the data from the orders table.

query = 'SELECT * FROM accounts JOIN orders ON accounts.id = orders.account_id'
query_to_df(query)

,id,name,website,lats,longs,primary_poc,sales_rep_id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18
3,4,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,1001,2016-01-02 01:18:24,144,32,0,176,718.56,239.68,0.00,958.24
4,5,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,1001,2016-02-01 19:27:27,108,29,28,165,538.92,217.21,227.36,983.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6907,6908,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970,4501,2016-06-29 04:03:39,11,199,59,269,54.89,1490.51,479.08,2024.48
6908,6909,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970,4501,2016-07-29 19:58:32,5,91,96,192,24.95,681.59,779.52,1486.06
6909,6910,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970,4501,2016-08-27 00:58:11,16,94,82,192,79.84,704.06,665.84,1449.74
6910,6911,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970,4501,2016-11-22 06:52:22,63,67,81,211,314.37,501.83,657.72,1473.92


Another way to select all columns from the two above tables is...

```
query = """SELECT orders.*, accounts.*
FROM accounts
JOIN orders
ON accounts.id = orders.account_id;"""

query_to_df(query)
```

Notice this result is the same as if you switched the tables in the `FROM` and `JOIN`. <br>Additionally, which side of the `=` a column is listed doesn't matter.

Personally, I think it makes sense to keep it uniform... <br>Meaning make the table at the left side of the `=` be the first table selected, while that at the right side be the second table.

In [21]:
# Try pulling standard_qty, gloss_qty, and poster_qty from the orders table, 
# and the website and the primary_poc from the accounts table.

query = 'SELECT orders.standard_qty, orders.gloss_qty, orders.poster_qty, \
        accounts.website, accounts.primary_poc FROM orders JOIN accounts ON \
        orders.account_id = accounts.id;'

query_to_df(query)

,standard_qty,gloss_qty,poster_qty,website,primary_poc
0,123,22,24,www.walmart.com,Tamara Tuma
1,190,41,57,www.walmart.com,Tamara Tuma
2,85,47,0,www.walmart.com,Tamara Tuma
3,144,32,0,www.walmart.com,Tamara Tuma
4,108,29,28,www.walmart.com,Tamara Tuma
...,...,...,...,...,...
6907,11,199,59,www.spartannash.com,Jewell Likes
6908,5,91,96,www.spartannash.com,Jewell Likes
6909,16,94,82,www.spartannash.com,Jewell Likes
6910,63,67,81,www.spartannash.com,Jewell Likes
